# Santa 2023 - initial solution

Many thanks to @whats2000 for explaining the problem and A* algorithm. 

This notebook heavily uses his code from https://www.kaggle.com/code/whats2000/a-star-algorithm-polytope-permutation/notebook, take a look and leave an upvote!

## 1. Imports & data loading

In [68]:
# essentials
import os
import pathlib
from copy import copy
import json
from pprint import pprint
import heapq
import time
import datetime

import pandas as pd
import numpy as np
from tqdm import tqdm

# visualisation
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
IN_KAGGLE = False

kaggle_folder = "/kaggle/input/"
local_folder = "./data/"
puzzle_info_df = pd.read_csv(kaggle_folder if IN_KAGGLE else local_folder + "santa-2023/puzzle_info.csv")
puzzles_df = pd.read_csv(kaggle_folder if IN_KAGGLE else local_folder + "santa-2023/puzzles.csv")
sample_submission_df = pd.read_csv(kaggle_folder if IN_KAGGLE else local_folder + "santa-2023/sample_submission.csv")


puzzle_info_df.head()

,puzzle_type,allowed_moves
0,cube_2/2/2,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
1,cube_3/3/3,"{'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ..."
2,cube_4/4/4,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
3,cube_5/5/5,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
4,cube_6/6/6,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."


In [70]:
puzzles_df.head()

,id,puzzle_type,solution_state,initial_state,num_wildcards
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0


In [87]:
puzzles_df[ puzzles_df['id'].isin([282, 281, 283]) ]

,id,puzzle_type,solution_state,initial_state,num_wildcards,general_puzzle_type,parsed_initial_state,parsed_solution_state
281,281,cube_33/33/33,A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;...,B;F;C;B;E;F;A;F;F;C;B;A;F;A;D;C;C;B;E;E;C;D;B;...,0,cube,"[B, F, C, B, E, F, A, F, F, C, B, A, F, A, D, ...","[A, A, A, A, A, A, A, A, A, A, A, A, A, A, A, ..."
282,282,cube_33/33/33,A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;...,F;B;A;F;A;C;B;B;C;B;B;A;E;F;A;E;E;E;B;E;B;F;E;...,0,cube,"[F, B, A, F, A, C, B, B, C, B, B, A, E, F, A, ...","[A, B, A, B, A, B, A, B, A, B, A, B, A, B, A, ..."
283,283,cube_33/33/33,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N1056;N4357;N3264;N3270;N924;N2183;N1095;N6526...,0,cube,"[N1056, N4357, N3264, N3270, N924, N2183, N109...","[N0, N1, N2, N3, N4, N5, N6, N7, N8, N9, N10, ..."


In [71]:
puzzles_df['puzzle_type'].value_counts()

puzzle_type
cube_3/3/3        120
cube_4/4/4         60
cube_5/5/5         35
cube_2/2/2         30
wreath_6/6         20
globe_3/4          15
wreath_7/7         15
cube_6/6/6         12
globe_1/8          10
wreath_12/12       10
globe_3/33          8
cube_10/10/10       5
cube_9/9/9          5
wreath_21/21        5
cube_8/8/8          5
globe_1/16          5
globe_2/6           5
cube_7/7/7          5
globe_6/4           5
globe_6/8           5
globe_6/10          5
cube_19/19/19       4
cube_33/33/33       3
wreath_33/33        3
globe_8/25          2
wreath_100/100      1
Name: count, dtype: int64

In [86]:
print(f"Total number of puzzles: {len(puzzles_df)}")

Total number of puzzles: 398


In [72]:
puzzles_df['general_puzzle_type'] = puzzles_df['puzzle_type'].apply(lambda x: x.split('_')[0])
puzzles_df['general_puzzle_type'].value_counts()

general_puzzle_type
cube      284
globe      60
wreath     54
Name: count, dtype: int64

> What are wildcards? 

In [73]:
puzzles_df.groupby(['puzzle_type'])['num_wildcards'].value_counts() \
    .unstack().fillna(0).astype('int').sort_values(0) \
    .style.background_gradient()

num_wildcards,0,2,4,6,8,10,12,16,18,34,38,42,54,176
puzzle_type,,,,,,,,,,,,,,
wreath_100/100,0,0,0,0,1,0,0,0,0,0,0,0,0,0
globe_6/10,1,1,0,1,2,0,0,0,0,0,0,0,0,0
wreath_33/33,2,0,0,0,1,0,0,0,0,0,0,0,0,0
globe_8/25,2,0,0,0,0,0,0,0,0,0,0,0,0,0
cube_19/19/19,3,0,0,0,0,0,0,0,0,0,0,0,0,1
cube_33/33/33,3,0,0,0,0,0,0,0,0,0,0,0,0,0
globe_2/6,3,1,1,0,0,0,0,0,0,0,0,0,0,0
cube_8/8/8,3,0,0,0,0,0,0,0,1,1,0,0,0,0
cube_9/9/9,3,0,0,0,0,0,0,1,0,0,1,0,0,0


In [74]:
sample_submission_df.head()

,id,moves
0,0,r1.-f1
1,1,f1.d0.-r0.-f1.-d0.-f1.d0.-r0.f0.-f1.-r0.f1.-d1...
2,2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...


## 2. Parse puzzle status

In [75]:
# Parsing the initial_state and solution_state columns
# Converting the semicolon-separated string values into lists of colors
puzzles_df['parsed_initial_state'] = puzzles_df['initial_state'].apply(lambda x: x.split(';'))
puzzles_df['parsed_solution_state'] = puzzles_df['solution_state'].apply(lambda x: x.split(';'))

# Displaying the modified dataframe with parsed states
puzzles_df[['id', 'puzzle_type', 'parsed_initial_state', 'parsed_solution_state']].head()

,id,puzzle_type,parsed_initial_state,parsed_solution_state
0,0,cube_2/2/2,"[D, E, D, A, E, B, A, B, C, A, C, A, D, C, D, ...","[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ..."
1,1,cube_2/2/2,"[D, E, C, B, B, E, F, A, F, D, B, F, F, E, B, ...","[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ..."
2,2,cube_2/2/2,"[E, F, C, C, F, A, D, D, B, B, A, F, E, B, C, ...","[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ..."
3,3,cube_2/2/2,"[A, C, E, C, F, D, E, D, A, A, F, A, B, D, B, ...","[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ..."
4,4,cube_2/2/2,"[E, D, E, D, A, E, F, B, A, C, F, D, F, D, C, ...","[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ..."


## 3. Parse allowed_moves

In [76]:
# Converting the string representation of allowed_moves to dictionary
puzzle_info_df['allowed_moves'] = puzzle_info_df['allowed_moves'].apply(lambda x: json.loads(x.replace("'", '"')))

# Selecting an example puzzle type and displaying its allowed moves
example_puzzle_type = puzzle_info_df['puzzle_type'].iloc[0]
example_allowed_moves = puzzle_info_df[puzzle_info_df['puzzle_type'] == example_puzzle_type]['allowed_moves'].iloc[0]

example_puzzle_type

'cube_2/2/2'

In [77]:
pd.DataFrame(example_allowed_moves)

,f0,f1,r0,r1,d0,d1
0,0,18,0,4,0,1
1,1,16,5,1,1,3
2,19,2,2,6,2,0
3,17,3,7,3,3,2
4,6,4,4,20,4,16
5,4,5,21,5,5,17
6,7,6,6,22,18,6
7,5,7,23,7,19,7
8,2,8,10,8,8,4
9,9,0,8,9,9,5


### How to read them?

> To read these tables

## Solving 2x2 Example

In [78]:
example_puzzle_type = puzzle_info_df['puzzle_type'].iloc[0]
example_allowed_moves = puzzle_info_df[puzzle_info_df['puzzle_type'] == example_puzzle_type]['allowed_moves'].iloc[0]

example_puzzle_type

'cube_2/2/2'

In [79]:
class PuzzleStateMachine:
    def __init__(self, puzzle_type, initial_state, solution_state, allowed_moves):
        print(f'Initializing puzzle state machine for {puzzle_type}.')
        self.puzzle_type = puzzle_type
        self.initial_state = initial_state
        self.solution_state = solution_state
        self.allowed_moves = allowed_moves
        self.state = initial_state
        self.history = [initial_state]

    def __repr__(self):
        return f'PuzzleStateMachine({self.puzzle_type}, {len(self.history)} moves)'
    
    def check_is_solved(self):
        return self.state == self.solution_state

    def check_allowed_move(self, move):
        if move not in self.allowed_moves.values():
            raise ValueError(f'Invalid move number {len(self.history)}: {move}')
    
    def inverse_move(self, move):
        return {v: k for k, v in enumerate(move)}
    
    def apply_move(self, move, inverse=False):
        """
        Apply a move or its inverse to the puzzle state.

        :param move: List representing the move as a permutation.
        :param inverse: Boolean indicating whether to apply the inverse of the move.
        """
        if inverse:
            inverse_move = self.inverse_move(move)
            self.state = [self.state[inverse_move[i]] for i in range(len(self.state))]
        else:
            self.state = [self.state[i] for i in move]
        self.history.append(self.state)
        
    def revert_move(self):
        if len(self.history) > 1:
            self.history.pop()
            self.state = self.history[-1]
        else:
            raise ValueError('Cannot revert initial state')

test_state = puzzles_df['parsed_initial_state'].iloc[0]
test_move = example_allowed_moves['f1']

puzzle = PuzzleStateMachine(
    example_puzzle_type, 
    test_state, 
    test_state, 
    example_allowed_moves
)


print('Initial state')
print(puzzle.state)

print(f"Test move:")
print(test_move)

puzzle.apply_move(test_move)
print('State after applying move')
print(puzzle.state)
puzzle.apply_move(test_move, inverse=True)
print('State after applying inverse move')
print(puzzle.state)

Initializing puzzle state machine for cube_2/2/2.
Initial state
['D', 'E', 'D', 'A', 'E', 'B', 'A', 'B', 'C', 'A', 'C', 'A', 'D', 'C', 'D', 'F', 'F', 'F', 'E', 'E', 'B', 'F', 'B', 'C']
Test move:
[18, 16, 2, 3, 4, 5, 6, 7, 8, 0, 10, 1, 13, 15, 12, 14, 22, 17, 23, 19, 20, 21, 11, 9]
State after applying move
['E', 'F', 'D', 'A', 'E', 'B', 'A', 'B', 'C', 'D', 'C', 'E', 'C', 'F', 'D', 'D', 'B', 'F', 'C', 'E', 'B', 'F', 'A', 'A']
State after applying inverse move
['D', 'E', 'D', 'A', 'E', 'B', 'A', 'B', 'C', 'A', 'C', 'A', 'D', 'C', 'D', 'F', 'F', 'F', 'E', 'E', 'B', 'F', 'B', 'C']


In [80]:
def heurestic(state, goal_state):
    """
    Heuristic function estimating the cost from the current state to the goal state.
    Here, we use the number of mismatched colors between the current state and the goal state.
    """
    return sum(s != g for s, g in zip(state, goal_state))

def apply_move(state, move, inverse=False):
    """
    Apply a move or its inverse to the puzzle state.

    :param state: List of colors representing the current state of the puzzle.
    :param move: List representing the move as a permutation.
    :param inverse: Boolean indicating whether to apply the inverse of the move.
    :return: New state of the puzzle after applying the move.
    """
    if inverse:
        # Creating a dictionary to map the original positions to the new positions
        inverse_move = {v: k for k, v in enumerate(move)}
        try:
            return [state[inverse_move[i]] for i in range(len(state))]
        except Exception as e:
            print(f"Len state: {len(state)}, len inverse_move: {len(inverse_move)}")
            raise e
    else:
        return [state[i] for i in move]
    
def a_star_search_with_timeout(puzzle, timeout=300):
    """
    A* search algorithm with a timeout feature.

    :param puzzle: Puzzle state machine object.
    :param timeout: The maximum time (in seconds) allowed for the search.
    :return: The shortest sequence of moves to solve the puzzle. Returns None if no solution is found.
    """

    # unpack puzzle object
    initial_state = puzzle.initial_state
    goal_state = puzzle.solution_state
    allowed_moves = puzzle.allowed_moves

    start_time = time.time()
    open_set = []  # priority queue of nodes to explore
    heapq.heappush(open_set, (0, initial_state, [])) # Each entry: (priority, state, path taken)

    closed_set = set() # set of states already explored

    while open_set:
        if time.time() - start_time > timeout:
            return None

        _, current_state, path = heapq.heappop(open_set)

        if current_state == goal_state:
            return path # solution found
        
        state_tuple = tuple(current_state)
        if state_tuple in closed_set:
            continue # skip already explored states

        closed_set.add(state_tuple)

        for move_name, move in allowed_moves.items():
            for inverse in [False, True]: # consider both move and its inverse
                new_state = apply_move(current_state, move, inverse)
                new_state_tuple = tuple(new_state)
                if new_state_tuple not in closed_set:
                    priority = len(path) + 1 + heurestic(new_state, goal_state)
                    heapq.heappush(open_set, (priority, new_state, path + [(move_name, inverse)]))

# testing with an example
                    
test_puzzle_type = puzzles_df['puzzle_type'].iloc[0]
test_initial_state = puzzles_df['parsed_initial_state'].iloc[0]
test_goal_state = puzzles_df['parsed_solution_state'].iloc[0]
test_allowed_moves = example_allowed_moves

test_puzzle = PuzzleStateMachine(
    test_puzzle_type, 
    test_initial_state, 
    test_goal_state, 
    test_allowed_moves
)

a_star_solution = a_star_search_with_timeout(
    test_puzzle,
)
print(f"A* solution: {a_star_solution}")

# testing solution with state machine
for move in a_star_solution:
    test_puzzle.apply_move(test_allowed_moves[move[0]], inverse=move[1])

test_puzzle.check_is_solved()

Initializing puzzle state machine for cube_2/2/2.
A* solution: [('r1', False), ('f1', True)]


True

In [81]:
# Modifying the A* search algorithm to improve efficiency
def improved_heuristic_with_wildcards(state, goal_state, num_wildcards):
    """
    Improved heuristic function considering wildcards.
    """
    mismatches = sum(s != g for s, g in zip(state, goal_state))
    return max(0, mismatches - num_wildcards)

def improved_a_star_search_with_wildcards(puzzle, num_wildcards, max_depth=30, timeout=100):
    """
    Improved A* search algorithm with wildcards, depth limit, and timeout.

    :param initial_state: List representing the initial state of the puzzle.
    :param goal_state: List representing the goal state of the puzzle.
    :param allowed_moves: Dictionary of allowed moves and their corresponding permutations.
    :param num_wildcards: Number of wildcards allowed for the puzzle.
    :param max_depth: Maximum depth to search to limit the search space.
    :param timeout: Time limit in seconds for the search.
    :return: Shortest sequence of moves to solve the puzzle, or None if no solution is found.
    """

    # unpack puzzle object
    initial_state = puzzle.initial_state
    goal_state = puzzle.solution_state
    allowed_moves = puzzle.allowed_moves
    
    start_time = time.time()
    open_set = []
    heapq.heappush(open_set, (0, initial_state, [], num_wildcards))  # (priority, state, path, remaining wildcards)
    closed_set = set()

    while open_set:
        if time.time() - start_time > timeout:
            return None  # Timeout

        _, current_state, path, remaining_wildcards = heapq.heappop(open_set)

        if len(path) > max_depth:  # Depth limit
            continue

        if current_state == goal_state or improved_heuristic_with_wildcards(current_state, goal_state, remaining_wildcards) == 0:
            return path

        closed_set.add((tuple(current_state), remaining_wildcards))

        for move_name, move in allowed_moves.items():
            for inverse in [False, True]:
                new_state = apply_move(current_state, move, inverse)
                if (tuple(new_state), remaining_wildcards) not in closed_set:
                    priority = len(path) + 1 + improved_heuristic_with_wildcards(new_state, goal_state, remaining_wildcards)
                    heapq.heappush(open_set, (priority, new_state, path + [(move_name, inverse)], remaining_wildcards))

    return None  # No solution found

# Running the improved A* search to find a solution
test_num_wildcards = puzzles_df['num_wildcards'].iloc[0]

new_test_puzzle = PuzzleStateMachine(
    test_puzzle_type, 
    test_initial_state, 
    test_goal_state, 
    test_allowed_moves
)
improved_a_star_solution = improved_a_star_search_with_wildcards(new_test_puzzle, test_num_wildcards)
print(f"Improved solution: {improved_a_star_solution}")  # Display the solution moves (if found)

for move in improved_a_star_solution:
    new_test_puzzle.apply_move(test_allowed_moves[move[0]], inverse=move[1])

new_test_puzzle.check_is_solved()

Initializing puzzle state machine for cube_2/2/2.
Improved solution: [('r1', False), ('f1', True)]


True

## Submission format

In [82]:
def format_solution_for_submission(puzzle_id, solution_moves):
    """
    Format the solution to a puzzle for submission.

    :param puzzle_id: The unique identifier of the puzzle.
    :param solution_moves: List of tuples representing the solution moves.
    :return: Formatted string suitable for submission.
    """
    formatted_moves = []
    for move, inverse in solution_moves:
        move_str = '-' + move if inverse else move
        formatted_moves.append(move_str)

    # Joining the moves into a single string separated by periods
    return {'id': puzzle_id, 'moves': '.'.join(formatted_moves)}

# Example: Formatting the solution for the first puzzle in the dataframe for submission
puzzle_id_example = puzzles_df['id'].iloc[0]
formatted_solution = format_solution_for_submission(puzzle_id_example, a_star_solution)
formatted_solution

{'id': 0, 'moves': 'r1.-f1'}

## Define solve function

In [83]:
def solve_puzzles(puzzles_df, puzzle_info_df, sample_submission_df, num_puzzles=None, limit_index=30):
    """
    Solve a set of puzzles using the A* search algorithm.

    :param puzzles_df: DataFrame containing puzzles.
    :param puzzle_info_df: DataFrame containing allowed moves for each puzzle type.
    :param sample_submission_df: DataFrame containing sample submission format.
    :param num_puzzles: Number of puzzles to solve (if None, solve all).
    :return: DataFrame with the solutions formatted for submission.
    """
    solutions = []

    # limit the number of puzzles if specified
    puzzles_to_solve = puzzles_df if num_puzzles is None else puzzles_df.head(num_puzzles)

    for index, row in tqdm(puzzles_to_solve.iterrows(), total=puzzles_to_solve.shape[0], desc="Solving Puzzles"):
        puzzle_id = row['id']
        initial_state = row['parsed_initial_state']
        solution_state = row['parsed_solution_state']
        puzzle_type = row['puzzle_type']
        num_wildcards = row['num_wildcards']
        allowed_moves = puzzle_info_df[puzzle_info_df['puzzle_type'] == puzzle_type]['allowed_moves'].iloc[0]

        solution_moves = None
        if index < limit_index:
            solution_moves = improved_a_star_search_with_wildcards(
                PuzzleStateMachine(puzzle_type, initial_state, solution_state, allowed_moves),
                num_wildcards
            )
        # if no solution found, 
        if solution_moves is None:
            solution_moves = sample_submission_df[
                sample_submission_df['id'] == puzzle_id
            ]['moves'].iloc[0].split('.')
            solution_moves = [(move.replace('-', ''), move.startswith('-')) for move in solution_moves]

        formatted_solution = format_solution_for_submission(puzzle_id, solution_moves)
        solutions.append(formatted_solution)

    return pd.DataFrame(solutions)

# Solving the puzzles
solved_puzzles_df = solve_puzzles(puzzles_df, puzzle_info_df, sample_submission_df)
solved_puzzles_df

Solving Puzzles:   0%|          | 0/398 [00:00<?, ?it/s]

Initializing puzzle state machine for cube_2/2/2.
Initializing puzzle state machine for cube_2/2/2.


Solving Puzzles:   1%|          | 2/398 [00:00<00:29, 13.54it/s]

Initializing puzzle state machine for cube_2/2/2.
Initializing puzzle state machine for cube_2/2/2.


Solving Puzzles:   1%|          | 4/398 [00:11<21:31,  3.28s/it]

...

Initializing puzzle state machine for cube_2/2/2.


Solving Puzzles:   7%|▋         | 29/398 [07:01<2:22:41, 23.20s/it]

Initializing puzzle state machine for cube_2/2/2.


Solving Puzzles: 100%|██████████| 398/398 [07:17<00:00,  1.10s/it] 


,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4,4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


## Submission

In [85]:
# Define the file path for the output CSV file
output_csv_path = 'submission.csv'
solved_puzzles_df.to_csv(output_csv_path, index=False)
print(f"Saved solution to {output_csv_path}")

Saved solution to submission.csv
